In [1]:
import joblib
import pandas as pd
from nltk.tokenize import sent_tokenize
from openai import OpenAI
import json
from counterfactual_inter_sbert import get_corrections_from_LLM_grammar, get_corrections_from_LLM_longer,get_prompt_grammar, get_prompt_longer

C:\Users\VICTUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
data = pd.read_csv('ASAP7 Test Set.tsv', sep='\t')
data.head()

,essay_id,essay,domain1_score
0,19392,My at saying your patient is. Your patient whe...,12
1,18926,Once upon a time there was a young girl named ...,18
2,18318,Patience is hard. You always @CAPS1 it now not...,14
3,18904,"Patience, @CAPS6 essential part of our world. ...",14
4,19131,One time I was pacient was when my mum said th...,8


In [4]:
# Sample 100 data to get corrections from OpenAI
data_100 = data[:100]
data_100.head()

,essay_id,essay,domain1_score
0,19392,My at saying your patient is. Your patient whe...,12
1,18926,Once upon a time there was a young girl named ...,18
2,18318,Patience is hard. You always @CAPS1 it now not...,14
3,18904,"Patience, @CAPS6 essential part of our world. ...",14
4,19131,One time I was pacient was when my mum said th...,8


In [18]:
essay = data_100.iloc[77]['essay']
essay

'I believe that computers are a great invention. It lets people interact with people from very far off places. Has lots of fun games you can play. And it is also a great way to learn information on topics and places very far off. Being able to talk to relatives from far away places is very important. @PERCENT1 of experts say that having a close knit family will give kids a greater chance of going to college. I understand why some people think "well why doesn\'t the " I grew up in a big family and just the whole day the phone was getting used. The computer has a lot of fun games that you can play. I see why you might think that the computer is why kids are over-weight But isn\'t the hour on the computer. Its eating habbits. Studies show that computers are use in the winter two time as much then. Going on the internet to learn new information is a great way to useing your souce. Just imagine this you straight A student comes home for the weekend about to do the biggest project lige forge

In [11]:
# ONLINE CORRECTIONS (REQUEST TO OPENAI's LLM)
essay = data_100.iloc[30]['essay']
model_name = 'gpt-4o-mini'
corrections_grammar = get_corrections_from_LLM_grammar(essay, model_name)
corrections_grammar

Requesting grammar corrections from LLM ... 


[{'original': 'half way',
  'correction': 'halfway',
  'type': 'Spelling',
  'sentence': 6},
 {'original': 'the passport were',
  'correction': 'the passports were',
  'type': 'Grammatical',
  'sentence': 8},
 {'original': 'a lot whining',
  'correction': 'a lot of whining',
  'type': 'Grammatical',
  'sentence': 7},
 {'original': 'for @LOCATION3 citizens',
  'correction': 'for the citizens of @LOCATION3',
  'type': 'Word Choice',
  'sentence': 4},
 {'original': 'we were coming',
  'correction': 'we were returning',
  'type': 'Word Choice',
  'sentence': 2}]

In [6]:
# Initialize an empty list to store the JSON responses
json_responses = []

# Iterate through each row in the DataFrame
for index, row in data_100.iterrows():
    try:
        # Extract the essay text
        essay = row['essay']
        # Get the JSON response from the function
        json_response = get_corrections_from_LLM_grammar(essay, model_name='gpt-4o')
        # Append the result to the list
        json_responses.append(json_response)
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        json_rsponses.append(None)  # Append None or an empty dict if an error occurs
    
    # Print progress every 10 iterations
    if (index + 1) % 10 == 0:
        print(f"Processed {index + 1} / {len(data_100)} essays")

Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Processed 10 / 100 essays
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Requesting grammar corrections from LLM ... 
Processed 20 / 100 essays
Requesting grammar corrections from LLM ... 
Req

In [7]:
len(json_responses)

100

In [8]:
joblib.dump(json_responses, 'files/json_responses_test_100')

['files/json_responses_test_100']

#### Error Corrections (eg. Corrections less than 5, which becomes a problem later)

In [9]:
corrections = joblib.load('files/json_responses_test_100')

In [12]:
corrections[30]

[{'original': 'half way',
  'correction': 'halfway',
  'type': 'Spelling',
  'sentence': 6},
 {'original': 'a lot whining',
  'correction': 'a lot of whining',
  'type': 'Grammatical',
  'sentence': 7},
 {'original': 'passport were',
  'correction': 'passports were',
  'type': 'Grammatical',
  'sentence': 8}]

In [13]:
corrections[30] = corrections_grammar

In [14]:
joblib.dump(corrections, 'files/json_responses_test_100')

['files/json_responses_test_100']

#### Check Missing Corrections (less than 5)

In [15]:
for i, inner in enumerate(corrections):
    if not isinstance(inner, (list, tuple)):
        print(f"Row {i}: not a list (type={type(inner)})")
    elif len(inner) < 5:
        print(f"Row {i}: length = {len(inner)}")